![](https://t1.daumcdn.net/cfile/tistory/9981C5345C35F75E2C)

**개요 :** 뱅크샐러드는 통합 자산조회, 맞춤형 자산관리, 금융상품 추천 등의 서비스를 제공하는 회사입니다. 

또 마이데이터 사업(표준화된 정보 제공 방식을 통해 데이터의 주체인 국민, 즉 개인이 스스로 정보를 제공하고 관리할 수 있도록 하는 것을 말합니다.)을 시작으로 정부의 데이터 산업 활성화 전략에 앞장서 많은 사람들이 금융을 쉽게 알고, 부담없이 접근할 수 있도록 노력하고 있으며, 이러한 일환으로 이번 대회를 후원하게 되었습니다. 

5회 대회는 1회 대회의 소상공인 신용카드 거래 내역 데이터를 사용하여 1회 대회에서 다 보여주지 못한 인사이트 발굴 및 시각화 대회를 진행합니다. 

### 패키지 설치

In [ ]:
### 패키지 설치 
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import plotly.graph_objs as go

import time
import random

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
### Permutation importance
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
###shap
import shap

## 목차
- 데이터 탐색
    - 결측치 제거 및 유니크 값
    - 시간에 따른 탐색
    - Amount의 심화 탐색
    - installments(할부) 심화 탐색
- 클러스터링 
    - 일별 거래의 변화율
    - 시간별 거래의 변화율
    - Prophet을 통한 클러스터 별 트렌드 분석
- 변수중요도 
    - Feature Engineering
    - Correlation
    - Tree Importance
    - Permutation Importance
    - Shap value

### 데이터 탐색

In [ ]:
train = pd.read_csv("train.csv",parse_dates=['date'])
train.shape

변수는 총 8개이고, 관측치는 330만개로 관측치에 비해 변수가 많이 적은 상황. 그리고 각 변수의 설명은 아래와 같음.

- store_id :각 파일에서의 상점 고유 번호.
- date : 거래 일자
- time :거래 시간
- card_id :카드 번호의 hash 값
- amount :매출액
- installments : 할부개월수. 일시불은 빈 문자열
- days_of_week : 요일, 월요일이 0, 일요일은 6
- holiday : 1이면 공휴일, 0이면 공휴일이 아님

In [ ]:
train.head()

데이터는 크게 3가지로 구분되어 짐. 시간, 식별화 코드, 매출
- 시간 : date, time, days_of_week, holiday
- 식별화 코드 : store_id, card_id
- 매출 : amount, installments

ㅡ> 참고 : 변수명 중에 holyday는 holiday가 오타난것 같음

아이디어
- row 1을 보면 amount가 음수인게 있는데 이는 취소된 거래를 말하는 것임. 바로 위 row 0의 거래가 3초만에 취소되었다는것을 의미. 
- date와 time의 경우 좀 더 구분의 편의를 위해서 년도/월/일/시간/분으로 변수를 확장 함. 
- days_of_week도 월~금, 토 ~ 일요일인지 여부를 나누어서 비교. 
- installments에 보면 결측치가 있는데, 이는 일시불을 의미하는것이므로 1으로 처리.

#### 결측치 제거

In [ ]:
#https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-nn-approach
def mis_value_graph(data):  
    data = [
    go.Bar(
        x = data.columns,
        y = data.isnull().sum(),
        name = 'Counts of Missing value',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
    layout= go.Layout(
        title= '"Total Missing Value By Column"',
        xaxis= dict(title='Columns', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Value Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='skin')
    
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color

mis_value_graph(train)

- Installments(할부) 변수에 약 334만개의 결측치가 있습니다. 이는 전체 데이터 336만개의 약 99.5%로 많은 비중을 차지하지만 데이터 설명에 따르면 installments의 결측치는 일시불을 의미하므로 1으로 채워 넣는것이 타당.
- 그 외의 변수에는 결측치가 없음. 

In [ ]:
### 결측치 처리 코드 
train['installments'] = train['installments'].fillna(1)

#### 유니크 갯수

In [ ]:
### 유니크 갯수 계산
train_unique = []
for i in train.columns.values:
    train_unique.append(len(train[i].unique()))
unique_train = pd.DataFrame()
unique_train['Columns'] = list(train.columns)
unique_train['Unique_value'] = train_unique

In [ ]:
data = [
    go.Bar(
        x = unique_train['Columns'],
        y = unique_train['Unique_value'],
        name = 'Unique value in features',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
layout= go.Layout(
        title= "Unique Value By Column",
        xaxis= dict(title='Columns', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Value Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='skin')

각 변수의 유니크 갯수
- store_id : 1775
- date : 730
- tiem : 85.924k
- card_id : 922.522k
- amount : 6031
- installments : 17
- days_of_week : 7
- holiday : 2

주목할만한 점
- store_id 는 1775개의 상점에 대해서 있다는 점.
- card_id 는 92만개로 high cardinarity 하다는 점.
- installments의 경우 17개로 다양한 할부 개월이 존재한다는 점.

In [ ]:
train['installments'].unique()

#### 시간에 따른 분석

In [ ]:
###요일변수
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

train['hour'] = train['time'].apply(lambda x: x[0:2])
train['hour'] = train['hour'].astype('int')

train['weekend'] = train['days_of_week'].apply(lambda x: 1 if x > 4 else 0)

In [ ]:
### 일별 Store_id의 구매건수
def scatter_plot(cnt_srs, color, showlegend1=False, name1 = ""):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        name = name1,
        showlegend=showlegend1,
        marker=dict(
            color=color,
        ),
    )
    return trace

train['date'] = pd.to_datetime(train['date'])

cnt_srs = train.groupby('date')['store_id'].size()


trace = scatter_plot(cnt_srs, 'red')

layout = go.Layout(
    height=600,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Dates in train set'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 카드 매출 내역 : 2016-08-01부터 2018-07-31까지 있는것을 확인할 수 있음 
- 2018년도 5월까지는 상승곡선을 보이지만 그 이후로는 하강곡선을 보이고 있음.
- 중간중간 거래건수가 확 떨어지는 날이 있음. 2016-09-15, 2017-01 28, 2017-10-04, 2018-02-16, 2018-05-06일.
    - 가설1 : 휴일이여서 거래건수가 확 떨어졌을 것이다. 
    
검증 : 휴일이여서 거래건수가 확 떨어졌을 것이다. 

In [ ]:
find_day = ['2016-09-15', '2017-01-28', '2017-10-04', '2018-02-16', '2018-05-06']
train1 = train[(train['date'] == '2016-09-15') | (train['date'] == '2017-01-28') | (train['date'] == '2017-10-04') | (train['date'] == '2018-02-16') | (train['date'] == '2018-05-06')]
train1.groupby('date').agg({'holyday':'mean'})

2016-09-15, 2017-01 28, 2017-10-04, 2018-02-16은 휴일이고 2018-05-06은 공휴일이 아님. 하지만 

- 2016-09-15은 추석으로 14,15,16,17,18일 총 5일을 쉬었던 연휴기간 
- 2017-01-28은 설날으로 27,28,29,30일 총 4일을 쉬었던 연휴기간 
- 2017-10-04는 추석으로 3,4,5,6,7,8,9일 총 7일을 쉬었던 연휴기간
- 2018-02-16은 설날으로 15,16,17일 총 3일을 쉬었던 연휴기간
- 2018-05-06은 공휴일은 아니었지만 5월5일 어린이날이 토요일로 주어진 까닭에 5월7일이 대체휴일이 되어서 5,6,7일 총 3일을 쉴 수 있었던 연휴기간

이러한 장기연휴기간들에 영향을 받아서 거래량이 감소. 

#### 일별 거래량의 정보를 주중, 주말, 요일에 휴일에 따라 세분화. 

In [ ]:

train1 = train[train['weekend']==0]
train2 = train[train['weekend']==1]

cnt_srs1 = train1.groupby('hour')['amount'].sum()
cnt_srs2 = train2.groupby('hour')['amount'].sum()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'weekday')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'weekend')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

<평일>

- 점심, 저녁시간에 카드 사용자 급증
- 16시-18시 -> 대체적으로 중고등학생 하교 시간과 가정주부들이 장보러 가는 시간에 맞춰 사용자수 증가추세
- 새벽시간대 amount의 합이 다른 시간대에 비해 가장 컸던 것과 달리 사용자수는 매우 적음

<주말>

- 점심부터 저녁시간에 걸쳐 완만한 그래프를 보임

In [ ]:
train1 = train[train['weekend']==0]
train2 = train[train['weekend']==1]

cnt_srs1 = train1.groupby('hour')['amount'].mean()
cnt_srs2 = train2.groupby('hour')['amount'].mean()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'weekday')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'weekend')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

평균적으로 주말과 평일의 경우 비슷한 트렌드를 가지고 있음. 평일(weekday)의 경우 상승과 하강폭이 주말(weekend)의 경우보다 크고, 값의 변동이 잦음. 

<평일>

- 22시부터 새벽시간에 결제건수(카드 id) 당 매출이 높음 -> 평일 직장인들 회식이나 대학생들 술자리 -> 밤~새벽 시간 카드 사용자는 대체적으로 직장인 또는 대학생일 확률이 높음
- 점심시간인 12시가 사용자수는 가장 높지만 결제건수 당 매출은 낮은편 -> 점심식사 더치페이 -> 역시 해당 시간대는 직장인 또는 대학생일 확률이 높음
- 10시-11시 -> 남편 출근이나 자녀 등교 후 브런치 식사 또는 카페에서 커피? -> 해당 시간대 카드 사용자는 가정주부일 확률이 높음
- 16시-18시 -> 중고등학생 하교시간, 남편 퇴근이나 자녀 하교 시간에 맞춰 대체적으로 가정주부들이 장보러 가는 시간 -> 중고등학생들이 하교 후 간식 등을 사먹지 않을까 의심 -> 해당 시간대 카드 사용자는 중고등학생과 가정주부일 확률이 높음

<주말>

- 새벽시간대를 제외하면 대체적으로 값이 비슷함
- 18시 이후 증가추세는 가족단위의 외식이거나 친구들과 저녁 약속 등? -> 학생부터 중장년층까지 전부 해당 가능하다고 보임

아래는 이를 더 세분화 한 요일별 시간에 따른 비교. 

In [ ]:
train_m = train[train['days_of_week']==0]
train_t1 = train[train['days_of_week']==1]
train_w = train[train['days_of_week']==2]
train_t2 = train[train['days_of_week']==3]
train_f = train[train['days_of_week']==4]
train_s1 = train[train['days_of_week']==5]
train_s2 = train[train['days_of_week']==6]

cnt_srs_m = train_m.groupby('hour')['amount'].mean()
cnt_srs_t1 = train_t1.groupby('hour')['amount'].mean()
cnt_srs_w = train_w.groupby('hour')['amount'].mean()
cnt_srs_t2 = train_t2.groupby('hour')['amount'].mean()
cnt_srs_f = train_f.groupby('hour')['amount'].mean()
cnt_srs_s1 = train_s1.groupby('hour')['amount'].mean()
cnt_srs_s2 = train_s2.groupby('hour')['amount'].mean()

trace_m = scatter_plot(cnt_srs_m, 'red', True ,'monday')
trace_t1 = scatter_plot(cnt_srs_t1, 'blue', True ,'tuesday')
trace_w = scatter_plot(cnt_srs_w, 'green', True ,'wendsday')
trace_t2 = scatter_plot(cnt_srs_t2, 'yellow', True ,'thursday')
trace_f = scatter_plot(cnt_srs_f, 'black', True ,'friday')
trace_s1 = scatter_plot(cnt_srs_s1, 'purple', True ,'saturday')
trace_s2 = scatter_plot(cnt_srs_s2, 'indigo', True ,'sunday')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace_m,trace_t1,trace_w,trace_t2,trace_f,trace_s1,trace_s2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 0시를 보면 화~토요일은 비슷한 시작을 보이지만, 일요일과 월요일은 매출액이 더 낮게 시작 함. 
- 5시부터는 전반적으로 비슷한 양상. 
- 토요일과 일요일은 8시이후로 평일보다 낮은 매출액을 기록함. 

In [ ]:
train1 = train[train['holyday']==0]
train2 = train[train['holyday']==1]

cnt_srs1 = train1.groupby('hour')['amount'].mean()
cnt_srs2 = train2.groupby('hour')['amount'].mean()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'holyday')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'non holyday')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

공휴일의 그래프는 평일 vs 주말의 그래프와 매우 비슷한 양상을 보임. 추가로 위에서 얻은 인사이트를 통해 휴가 전날의 경우에 대해 분석해보도록 하겠음. 

In [ ]:
ex = train[train['holyday']==1]
holiday_list = ex.date.unique()
d = np.timedelta64(1, 'D')
type(d)
before_list = holiday_list - d

train1 = train[(train["date"].isin(holiday_list))|(train['days_of_week']==5 )]
train2 = train[~(train["date"].isin(holiday_list))&(train['days_of_week']!=5 )]

cnt_srs1 = train1.groupby('hour')['amount'].mean()
cnt_srs2 = train2.groupby('hour')['amount'].mean()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'party_tonight')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'tomorrow_work')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 공휴일전날과 내일 출근을 하는 날을 비교해보면 바로 위에서 그렸던 그래프와 색깔만 반대인것을 확인할 수 있음. 
- 기존의 데이터 탐색대로, 휴일의 경우가 중요한것이 아닌 다음날이 쉬는날인지 여부에 따라 매출액의 변동이 달라지는것을 알 수 있음.

그 이유는 store에서 소비를 하는 소비층이 대부분 직장이기 떄문에 다음날이 휴일인지 여부에 따라 민감할 수 밖에 없어서임. 

#### Amount(매출액) 분석

In [ ]:
### Amount 에 대한 분석
train['amount'].describe()

In [ ]:
## histogram
trace1 = go.Histogram(
    x=train['amount'],
    histnorm='percent',
    name='control',
    xbins=dict(
        start=-10000,
        end=10000,
        size=200
    ),
    marker=dict(
        color='#FFD7E9',
    ),
    opacity=0.75
)

data = [trace1]

layout = go.Layout(
    title='Sampled Results',
    xaxis=dict(
        title='Value'
    ),
    yaxis=dict(
        title='Count'
    )
    #,
    #bargap=0.2,
    #bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

- amount의 경우 0~2000까지는 반쪽짜리 정규분포모습을 보임. 
- 대부분의 값이 0~200사이에 몰려 있음. 
- 0인 값과 음수인 값도 존재하는데, 이는 거래가 취소되거나 표기가 실수된 경우. 

#### 거래가 취소 된 경우 분석. 

In [ ]:
print(train[train['amount']<0].shape[0],train[train['amount']==0].shape[0])

Amount의 이상치의 수
- amount가 0보다 이하로 취소 된 경우는 33832건
- 거래의 표기실수는 1024건 

실제로 거래가 취소 된 케이스는 두가지
- 거래가 취소되고 재거래가 발생하지 않음. 
- 거래가 취소되고 재거래가 발생. 

거래가 취소되고 재거래가 발생하지 않음. 

In [ ]:
train.head(3)

거래가 취소되고 재거래가 발생. 

In [ ]:
train.iloc[106:113,]

106번째, 11번째, 112번쨰 row를 보면 동일한 card_id를 가지고 동일한 할부개월수를 긁은 것을 볼 수 있음. 처음에는 108에 대한 amount가 나왔지만 108을 취소하고 89로 다시 결제했다고 보는것이 타당. 

추가로 결제의 취소에 대해 시간대와 상점, 날짜별 표를 살펴보면 아래와 같음. 

In [ ]:
cancel_df = train[train['amount']<0]
cancel_df_hour = pd.DataFrame(cancel_df.groupby('hour').size().reset_index())
cancel_df_hour.columns=['hour','hour_size']

cnt_srs = cancel_df_hour['hour_size']


trace = scatter_plot(cnt_srs, 'red')

layout = go.Layout(
    height=600,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Dates in train set'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 12시, 13시, 19시가 다른 시간대에 비해서 높으며, 주로 낮에서 저녁 사이의 시간대에 거래 취소가 많은 것을 확인할 수 있음
- 그중에서도 주로 식사시간에 거래취소를 많이 하는 편. 학생이나 직장인의 경우에는 식사시간이나 일과가 끝난 후에 거래취소를 위해 가게를 방문하는 경우가 많기 때문에 이 시간대에 거래 취소가 많은 것으로 보인다. 또한 이 시간대에 거래건수도 많기 때문에 거래와 취소가 빈번하게 일어나는 시간대로 볼 수 있음.

다음으로, 거래취소건수와 전체 거래수 대비 거래취소건수 비율을 함께 살펴볼 필요가 있음. 전체 거래수가 적다면 거래 취소 건수도 그에 대비해서 적을 수 있기 때문. 

In [ ]:
cancel_df_hour1 = pd.merge(cancel_df_hour,train.groupby('hour').size().reset_index())
cancel_df_hour1.columns = ['hour','cancel_size','total_size']
cancel_df_hour1['cancel/total'] = cancel_df_hour1.cancel_size/cancel_df_hour1.total_size
cnt_srs2 = cancel_df_hour1['cancel/total']


trace = scatter_plot(cnt_srs2, 'red')

layout = go.Layout(
    height=600,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Dates in train set'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 전체 구매 수 대비 거래를 취소한 수를 살펴보았을 때는 위의 그래프를 보면 알 수 있듯이 아침 시간대에서 그 비율이 높은 것을 확인할 수 있음. 그 이유는 여러개가 있겠지만, 가게의 오픈시간이라든지 저녁에 집에와서 거래가 잘 못된것을 알고 다음날 찾아갔을 확률이 높음. 추가로 아침에는 거래의 빈도가 적어서 비율이 높게 나온것이기도 함. 

In [ ]:
store_cancel = cancel_df.groupby('store_id').size().reset_index()
store_cancel.columns = ['store_id','cancel_size']
store_cancel = store_cancel.sort_values(by='cancel_size',ascending=False).head(10)
store_cancel = store_cancel.reset_index(drop=True)

data = [
    go.Bar(
        x = store_cancel['store_id'],
        y = store_cancel['cancel_size'],
        name = 'Top 10 : cancel_size of store_id ',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
layout= go.Layout(
        title= "Cancel Size By Column",
        xaxis= dict(title='Store_id', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Cancel Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='skin')

- store_id가 161, 1024인 상점은 다른 상점들에 비해서 거래 취소 건수가 많음. 거래 취소가 많다는 것은 거래 수가 많을 수 있지만 실제 매출액으로 계산될 거래 수는 그만큼 적어진다는 의미가 된다. 따라서 거래 취소가 많은 상점은 더 자세히 살펴볼 필요가 있음.

거래취소건수와 전체 거래수 대비 거래취소건수 비율

In [ ]:
store_cancel = cancel_df.groupby('store_id').size().reset_index()
store_cancel.columns = ['store_id','cancel_size']
store_cancel = pd.merge(store_cancel,train.groupby('store_id').size().reset_index())
store_cancel.columns = ['store_id','cancel_size','total_size']
store_cancel['cancel/total'] = 2 * store_cancel.cancel_size/store_cancel.total_size * 100
store_cancel = store_cancel.sort_values(by='cancel/total',ascending=False).head(10).reset_index(drop=True)

data = [
    go.Bar(
        x = store_cancel['store_id'],
        y = store_cancel['cancel/total'],
        name = 'Top 10 : cancel_size ratio of store_id ',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
layout= go.Layout(
        title= "Cancel Size By Column",
        xaxis= dict(title='Store_id', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Cancel ratio (%)', ticklen=5, gridwidth=2),
        showlegend=True
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='skin')

스토어별도 전체거래수 대비 거래 취소 건수의 비율 상위 10개를 살펴봄. 
- 앞서 거래 취소 건수가 많았던 161번 가게는 전체거래수 대비 거래취소건수의 비율도 높은 편에 속했음. 
- 반면에 1024번 가게는 단순히 거래를 취소한 건수는 높았지만, 전체거래수에 대비했을 때는 161번 가게보다는 거래수를 신뢰할 수 있음.
- 전체 거래수 대비 거래 취소 건수 비율은 주로 전체 거래수가 적은 곳에서 높았다. 거래취소한 건수는 적지만 전체 거래수도 적기 때문에 866, 988, 350, 1439 등 거래의 절반이상이 취소인 경우에는 해당 가게의 거래수를 신뢰하기 어렵다고 할 수 있다. 심지어 866, 988, 350가게들은 모든 거래를 취소한 경우.

#### installments(할부) 분석
할부가 일시불일 때(빨간 색)과 일시불이 아닐 때, 즉 할부를 했을 때(파란색) 두 경우를 나누어 매출액을 시간 별 평균 비교.

In [ ]:
def scatter_plot(cnt_srs, color, showlegend1=False, name1 = ""):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        name = name1,
        showlegend=showlegend1,
        marker=dict(
            color=color,
        ),
    )
    return trace

df1 = train[train['amount']>0]
train1 = df1[df1['installments']==1]
train2 = df1[df1['installments']!=1]

cnt_srs1 = train1.groupby('hour')['amount'].mean()
cnt_srs2 = train2.groupby('hour')['amount'].mean()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'no installment')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'installments')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    #title='sum installments in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 할부를 했을 때의 매출액이 일시불일 때의 매출액보다 어느 시간대에서든 크게 상회하는 것을 볼 수 있음. 이를 통해 확실히 큰 금액을 결제할 때 할부를 선택한다는 것을 확인할 수 있음.
- 또 일시불일 때는 어떤 시간대이든 매출액이 낮게 유지되는 반면 할부를 했을 때는 시간별 매출액의 변동이 큼. 이를 통해 일시불로 결제되는 금액은 상대적으로 한정적이고 작은 금액임을 확인할 수 있음. 

주중(빨강), 주말(파랑)에 따른 할부 개월 수의 추이를 시간 별 평균 비교. 

In [ ]:
train1 = df1[df1['weekend']==0]
train2 = df1[df1['weekend']==1]

cnt_srs1 = train1.groupby('hour')['installments'].mean()
cnt_srs2 = train2.groupby('hour')['installments'].mean()

trace1 = scatter_plot(cnt_srs1, 'red', True ,'weekday')
trace2 = scatter_plot(cnt_srs2, 'blue', True ,'weekend')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    #title='sum installments in train set',
    #legend = ['weekday','weekend']
    #showlegend=True
    #label=['weekday','weekend']
)

data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- 주중의 경우 새벽 시간대를 제외하고 대부분의 시간대에서 주말의 할부 개월 수를 상회하는 모습. 즉 사람들이 주중에 할부를 더 많이 선택하고 할부 개월 수를 크게 둔다는 것을 확인할 수 있음. 

### 클러스터링
- 일별 거래량의 변화율이 비슷한 상점끼리 묶어서 분석을 진행 : 주기를 전날과 7일전으로 하여 매주 주기성이 얼마나 다른지를 확인하여 클러스터 진행. 
- 시간 별 거래량의 변화율이 비슷한 상점끼리 묶어서 클러스터 진행.

In [ ]:
### 주기를 전날과 7일전으로 하여 
train1 = train.groupby(['store_id','date'])['amount'].agg({'amount':'mean'}).reset_index()
train1['amount_lag1'] = train1.groupby(['store_id'])['amount'].shift(1)
train1['amount_lag7'] = train1.groupby(['store_id'])['amount'].shift(7)
train1 = train1.fillna(0)

train1['amount_ratio1'] = train1['amount']/ (train1['amount_lag1']+1)
train1['amount_ratio7'] = train1['amount']/ (train1['amount_lag7']+1)
train1 = train1[train1['date']>='2017-01-01']

train2 = train1.groupby(['store_id']).agg({'amount_ratio1':'mean','amount_ratio7':'mean'}).reset_index()#feature = train1[['store_id','amount_ratio1','amount_ratio7','amount_ratio14']]
feature = train2[['store_id','amount_ratio1','amount_ratio7']]

plt.scatter(train2['amount_ratio1'],train2['amount_ratio7'])

보면 Store_id 847하나가 동 떨어져 있어서 이를 제거하고 다시 클러스터 진행. 

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt


feature = feature[feature['store_id']!=847]

Z = linkage(feature,'ward')

plt.figure(figsize=(50,20))
dendrogram(Z,
           leaf_rotation=100,
           leaf_font_size=1,
)
plt.show()

위의 결과에 추가적으로 847번쨰 store를 넣고 클러스터별로 요일별 거래량을 보면 아래와 같음. 

In [ ]:
from scipy.cluster.hierarchy import fcluster

k=3
predict=fcluster(Z, k, criterion='maxclust')
feature['cluster']=predict
train1 = pd.merge(train1,feature,on='store_id', how='left')
train1 = train1.fillna(0)

train_0 = train1[train1['cluster']==0]
train_1 = train1[train1['cluster']==1]
train_2 = train1[train1['cluster']==2]
train_3 = train1[train1['cluster']==3]
#train_4 = train1[train1['cluster']==4]

cnt_srs_0 = train_0.groupby('date')['amount'].mean()
cnt_srs_1 = train_1.groupby('date')['amount'].mean()
cnt_srs_2 = train_2.groupby('date')['amount'].mean()
cnt_srs_3 = train_3.groupby('date')['amount'].mean()
#cnt_srs_4 = train_4.groupby('date')['amount'].mean()

trace_0 = scatter_plot(cnt_srs_0, 'red', True ,'Cluster_0')
trace_1 = scatter_plot(cnt_srs_1, 'blue', True ,'Cluster_1')
trace_2 = scatter_plot(cnt_srs_2, 'green', True ,'Cluster_2')
trace_3 = scatter_plot(cnt_srs_3, 'black', True ,'Cluster_3')
#trace_4 = scatter_plot(cnt_srs_4, 'indigo', True ,'Cluster_4')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set '
)

data = [trace_0,trace_1,trace_2,trace_3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

- Cluster_0은 Store_id 847로 매일 거래가 이루어진 것이 아니라 특정 기간에만 이루어지고 매우 많은 매출액을 남겼음. 
- Cluster_1은 2와 3에 비해 어떤 이유에서인지 날짜 별 등락 폭이 굉장히 큼.
- Cluster_2는 아주 안정적인 매출 평균을 보여주는 집단
- Cluster_3 은 비교적 안정적인 매출을 보여주고 있지만, 특정 날짜에 매출이 확 줄거나, 또는 매출이 급등하는 등의 모습을 보여주고 있음, 예를 들어 2017/05/20에 급락하거나, 2017/05/30에 급등하는 모습을 보여주는 집단임.(음수의 값을 가짐.)

추가적으로 시간별 거래량의 변화율이 비슷한 store끼리 묶어서 클러스터 진행.

In [ ]:
### 주기를 전날과 7일전으로 하여 
train1 = train.groupby(['store_id','hour'])['amount'].agg({'amount':'mean'}).reset_index()

for i in range(1,24):
    train1['amount_lag{}'.format(i)] = train1.groupby(['store_id'])['amount'].shift(i)
    train1 = train1.fillna(0)
    train1['amount_ratio{}'.format(i)] = train1['amount']/ (train1['amount_lag{}'.format(i)]+1)
    del train1['amount_lag{}'.format(i)]
train1.head()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

excluded_features = ['store_id','amount','hour']
features  = [_f for _f in train1.columns if _f not in excluded_features]


train2 = train1[train1['hour']==0].reset_index(drop=True)
store_id = train2['store_id']
train2 = train2[features]

scaler = StandardScaler()
model = KMeans(n_clusters=3)

pipeline = make_pipeline(scaler,model)
pipeline.fit(train2)

predict = pd.DataFrame(pipeline.predict(train2))

ks = range(1,10)

inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(train2)
    inertias.append(model.inertia_)

# Plot ks vs inertias
plt.figure(figsize=(20,10))
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

시간별 거래량의 비율에 대한 K-menas 클러스터의 Eblow method를 적용한 결과 최적은 2~4dml k로 보임. 최대한 자세한 비굘르 하기 위해서 k=4로 설정하고 분석. 

In [ ]:
scaler = StandardScaler()
model = KMeans(n_clusters=4)

pipeline = make_pipeline(scaler,model)
pipeline.fit(train2)

predict = pd.DataFrame(pipeline.predict(train2))
train2['cluster'] = predict
train2['store_id'] = store_id

In [ ]:
store = train2[['store_id','cluster']]
train1 = pd.merge(train1,store,how='left',on='store_id')

In [ ]:
from scipy.cluster.hierarchy import fcluster

train_0 = train1[train1['cluster']==0]
train_1 = train1[train1['cluster']==1]
train_2 = train1[train1['cluster']==2]
train_3 = train1[train1['cluster']==3]
#train_4 = train1[train1['cluster']==4]

cnt_srs_0 = train_0.groupby('hour')['amount'].mean()
cnt_srs_1 = train_1.groupby('hour')['amount'].mean()
cnt_srs_2 = train_2.groupby('hour')['amount'].mean()
cnt_srs_3 = train_3.groupby('hour')['amount'].mean()
#cnt_srs_4 = train_4.groupby('date')['amount'].mean()

trace_0 = scatter_plot(cnt_srs_0, 'red', True ,'Cluster_0')
trace_1 = scatter_plot(cnt_srs_1, 'blue', True ,'Cluster_1')
trace_2 = scatter_plot(cnt_srs_2, 'green', True ,'Cluster_2')
trace_3 = scatter_plot(cnt_srs_3, 'black', True ,'Cluster_3')
#trace_4 = scatter_plot(cnt_srs_4, 'indigo', True ,'Cluster_4')

layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='sum amount in train set '
)

data = [trace_0,trace_1,trace_2,trace_3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

y값이 다르기 때문에 분석이 힘드므로 1시방향의 클러스터를 모두 제거하고, 하나씩 넣어서 분석하도록 하겠음. (1시 방향의 cluster0~3을 모두 눌러서 없앤 후에 아래의 순서에 따라 하나씩 클러스터번호를 누르면 하나씩 비교가 가능.)

- Cluster0는 아침, 점심, 저녁 3가지 시간대에 가장 활발 한 거래를 보이는 전형적인 음식점 형태임. 
- Cluster1는 새벽과 오후에만 거래가 활발한 술집의 모습을 보여줌. (7~19시 까지의 거래는 거의 0에 가까움. )
- Cluster2는 아침, 점심, 16시 , 저녁시간에 활발하고 그 뒤로 거래가 뚝 감소했다가 증가하는 패턴을 보임.
- Cluster3는 5시, 10시, 13시, 22시에 피크를 찍고 하강과 상승을 반복하는 모습. 

위에서 했던 일별 거래패턴이 비슷한 클러스터4가지를 Prophet에 적용해 트렌드를 파악하도록 하겠음. 

In [ ]:
from datetime import date, timedelta
from fbprophet import Prophet

각각 클러스터의 대푯값으로 하나씩 뽑아서 진행. 
- Cluster 1 : 44	
- Cluster 2 : 0
- Cluster 3 : 1799

Cluster0은 847로 제외.

In [ ]:
train = pd.read_csv("train.csv", parse_dates=['date'])

Holiday = list(train[train['holyday']==1]['date'].unique())
HOLIDAY = pd.DataFrame()
HOLIDAY['ds'] = Holiday
HOLIDAY['holiday'] = '휴일'
HOLIDAY = HOLIDAY.sort_values('ds')

df = train[train['store_id'] == 44]
amount = df.loc[:,['date','amount']] 
amount = amount.groupby('date', as_index=False).sum() 
amount['date'] = pd.to_datetime(amount['date'])
amount = amount.rename(columns = {'date': 'ds', 'amount': 'y'})

p = amount.set_index('ds').plot(figsize = (12, 5)) 
p.set_ylabel('daily amount of store id 44')
p.set_xlabel('date')
plt.show

In [ ]:
model = Prophet(interval_width = 0.95,holidays=HOLIDAY) #신뢰구간 95%
model.fit(amount) 

future = model.make_future_dataframe(periods = 100) #데이터 내 마지막 날짜로부터 향후 100일까지 확장

#prediction
forecast = model.predict(future)
model.plot_components(forecast) 

클러스터1의 경우를 보면
- 트렌드는 지속적으로 하락.
- 추석의 경우 거래가 감소
- 일요일부터 화요일까지 거래가 감소하다가 수요일날 증가, 다시 감소하여 최하를 찍고 상승

In [ ]:
df = train[train['store_id'] == 0]
amount = df.loc[:,['date','amount']] 
amount = amount.groupby('date', as_index=False).sum() 
amount['date'] = pd.to_datetime(amount['date'])
amount = amount.rename(columns = {'date': 'ds', 'amount': 'y'})

p = amount.set_index('ds').plot(figsize = (12, 5)) 
p.set_ylabel('daily amount of store id 0')
p.set_xlabel('date')
plt.show

In [ ]:
model = Prophet(interval_width = 0.95,holidays=HOLIDAY) #신뢰구간 95%
model.fit(amount) 

future = model.make_future_dataframe(periods = 100) #데이터 내 마지막 날짜로부터 향후 100일까지 확장

#prediction
forecast = model.predict(future)
model.plot_components(forecast) 

클러스터2의 경우를 보면
- 트렌드는 지속적으로 하락.
- 휴일의 경우 전반적으로 소비가 증가. 
- 월요일을 최하점으로 점점 거래가 상승함. 

In [ ]:
df = train[train['store_id'] == 1799]
amount = df.loc[:,['date','amount']] 
amount = amount.groupby('date', as_index=False).sum() 
amount['date'] = pd.to_datetime(amount['date'])
amount = amount.rename(columns = {'date': 'ds', 'amount': 'y'})

p = amount.set_index('ds').plot(figsize = (12, 5)) 
p.set_ylabel('daily amount of store id 1799')
p.set_xlabel('date')
plt.show

In [ ]:
model = Prophet(interval_width = 0.95,holidays=HOLIDAY) #신뢰구간 95%
model.fit(amount) 

future = model.make_future_dataframe(periods = 100) #데이터 내 마지막 날짜로부터 향후 100일까지 확장

#prediction
forecast = model.predict(future)
model.plot_components(forecast) 

클러스터3의 경우를 보면
- 트렌드는 지속적으로 하락.
- 휴일의 경우 전반적으로 소비가 증가. 
- 월요일과 금요일을 최하점으로 나머지는 비슷한 모습을 보임. 

### 변수중요도 

In [ ]:
train = pd.read_csv("train.csv",parse_dates=['date'])
train = train.fillna(1)

Feature Engineering
- 요일
- 이번 달 평균 거래 횟수, 최소 매출액, 평균 매출액, 최대 매출액
- N일 전 대비 매출액 변동 (1, 3, 7일)
- 지난 3일의 평균 매출액

In [ ]:
### 요일
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

train['hour'] = train['time'].apply(lambda x: x[0:2])
train['hour'] = train['hour'].astype('int')

train['weekend'] = train['days_of_week'].apply(lambda x: 1 if x > 4 else 0)

In [ ]:
### 이번 달 평균 거래 횟수, 최소 매출액, 평균 매출액, 최대 매출액
train1 = train.groupby(['store_id','month'])['installments'].agg({'count','max','mean','min'}).reset_index()
train1.columns = ['store_id','month','i_m_cnt','i_m_max','i_m_mean','i_m_min']
train = pd.merge(train,train1,on=['store_id','month'],how='left')

### 하루 평균 거래 횟수, 최소 매출액, 평균 매출액, 최대 매출액
train1 = train.groupby(['store_id','date'])['installments'].agg({'count','max','mean','min'}).reset_index()
train1.columns = ['store_id','date','i_d_cnt','i_d_max','i_d_mean','i_d_min']
train = pd.merge(train,train1,on=['store_id','date'],how='left')

train.head()

In [ ]:
### N일 전 대비 매출액 변동 (1, 2, 3, 7일)
train['lag1_i_d_mean'] = train.groupby(['store_id'])['i_d_mean'].shift(1)
train['lag2_i_d_mean'] = train.groupby(['store_id'])['i_d_mean'].shift(2)
train['lag3_i_d_mean'] = train.groupby(['store_id'])['i_d_mean'].shift(3)
train['lag7_i_d_mean'] = train.groupby(['store_id'])['i_d_mean'].shift(7)

### 지난 3일의 평균 매출액
train['Mv3_i_d_mean'] = (train['lag1_i_d_mean']+train['lag2_i_d_mean']+train['lag3_i_d_mean'])/3

train['lag1_i_d_mean'] = train['i_d_mean']/train['lag1_i_d_mean']
train['lag2_i_d_mean'] = train['i_d_mean']/train['lag2_i_d_mean']
train['lag3_i_d_mean'] = train['i_d_mean']/train['lag3_i_d_mean']
train['lag7_i_d_mean'] = train['i_d_mean']/train['lag7_i_d_mean']
train.head()

변수중요도
- Correlation
- Feature importances of Tree models
- Permutation importance
- SHAP values

In [ ]:
train1 = train.dropna(axis=0)
cols_to_drop = ['store_id', 'date', 'time', 'card_id']
cols_to_fit = [col for col in train1.columns if col not in cols_to_drop]

corr = train1[cols_to_fit].corr()

plt.figure(figsize=(18,10))
sns.heatmap(
    corr,
    xticklabels=corr.columns.values,
    yticklabels=corr.columns.values,
    annot=True,
    linecolor='white',
    linewidths=0.1,
    cmap="RdBu"
)
plt.show()

상관관계로 살펴봤을때, 의미있는 관계는

- i_d_min - installments
- i_d_cnt - i_m_cnt
- i_d_min - i_d_max
- i_d_mean - Mv3_i_d_mean
- lag3_i_d_mean - lag2_i_d_mean

#### Feature importance of Tree models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

target = 'i_d_mean'
cols_to_drop = ['date','time','card_id', target]
cols_to_fit = [col for col in train.columns if col not in cols_to_drop]
train1, val1 = train_test_split(train, test_size=0.3, random_state=0)

from lightgbm import LGBMRegressor
params = {
    'n_estimators': 100,
    'learning_rate': 0.3, 
    'num_leaves': 20,
    'objective': 'regression_l2', 
    'metric': 'mae',
    'verbose': -1,
}

model = LGBMRegressor(**params)
model.fit(
    train1[cols_to_fit], train1[target],
    eval_set=[(val1[cols_to_fit], val1[target])],
    eval_metric='mae',
    verbose=20,
)

feature_importance = pd.DataFrame(sorted(zip(model.feature_importances_, cols_to_fit)), columns=['Value','Feature'])

plt.figure(figsize=(10, 6))
sns.barplot(x="Value", y="Feature", data=feature_importance.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()

변수중요도로 3일의 평균 Moving Average와 평균 매출액의 상승비율이 많이 연관되어 있는것을 볼 수 있음. 추가적으로 min, mean과 같은 통계적인 값들도 유의미함을 확인할 수 있음.

#### Permutation importance


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

train1 = train.fillna(-1)
train1, val1 = train_test_split(train1, test_size=0.3, random_state=0)

perm = PermutationImportance(model, random_state=42).fit(val1[cols_to_fit], val1[target])
eli5.show_weights(perm, feature_names=list(cols_to_fit))

#### SHAP values

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(val1[cols_to_fit])
shap.summary_plot(shap_values, val1[cols_to_fit], plot_type='bar')

In [ ]:
shap.summary_plot(shap_values, val1[cols_to_fit], feature_names=cols_to_fit)

변수중요도 측면에서 각자가 우선시로 여기는것은 좀 다르지만, 전반적으로 통계값(min, cnt, max)과 lag의 값들이 중요한것을 파악할 수 있음.

결론 : 데이터 탐색을 통해서 살펴 본 결과 Store_id의 경우 주중, 주말, 휴일에 따라서 다른 형태를 보이는것을 확인할 수 있었고 일별 시간별에 따라 4가지 유형의 클러스터로 나뉘는 것을 확인할 수 있었음. 마지막으로 파생변수 생성과 변수중요도를 통해서 Store의 일 평균 매출액을 예측할 때 중요한지를 살펴봄으로써 대출의 한도를 선정할 수 있을거라 판단 됨. 